# Web Scraping for MDE data

In this note book we will explore downloading Minnesota Department of Education data and looking up the ACT code for schools missing that code. For this script you will need to have the packages below as well as a geckodriver with a path available to where this script is being executed. A web browser will be launched to select the actions needed to acquire the department of education data and Selenium will be used with a Python wrapper. 

In [1]:
## Read into memory the classes and functions needed

from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
import time
import os
from socket import socket
import pandas as pd
import wget
import datetime
import shutil
#import messytables

In [2]:
# Set the preferences for the firefox web browser
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.manager.showWhenStarting', False)
fp.set_preference('browser.download.dir', '/tmp')
fp.set_preference("http.response.timeout", 300)
fp.set_preference("dom.max_script_run_time", 300)
fp.set_preference('webdriver.load.strategy', 'unstable')

#fp.set_preference("browser.helperApps.neverAsk.openFile", "application/octet-stream");
#fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream");

fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
fp.update_preferences()

In [3]:
# MDE web app for hs enrollment data
searchURL = 'http://w20.education.state.mn.us/MDEAnalytics/Data.jsp'

In [4]:
# Launch web browser and navigate to the searchURL
driver = webdriver.Firefox(fp)
driver.get(searchURL)

In [5]:
# switch into the frame titled 'floatframe'
driver.switch_to.frame(driver.find_element_by_name('floatframe'))

In [6]:
# find the element by xpath that has the text 'Student'
ids_a = driver.find_elements_by_xpath("//*[text()='Student']")

In [7]:
# extract the link into an object
for ii in ids_a:
    link = ii.get_attribute('href')
time.sleep(4)

In [8]:
#open the link to Student level data reporting on the MDE site
driver.get(link)
time.sleep(3)

In [9]:
# Switch into the frame titled 'floatframe'
time.sleep(10)
driver.switch_to.frame(driver.find_element_by_name('floatframe'))

In [10]:
# Click the button to display all data reporting around Students
driver.find_element_by_xpath("//input[@id='button1' and @name='button1']").click()

In [11]:
# Switch into the frame from the output of the button click
driver.switch_to.frame(driver.find_element_by_name('Report'))

## Create new folders if they don't already exist for MDE, or however many states. 

Can also keep everything in one folder as well.

In [12]:
newpath_main = r'C:\MDE_Data'
if not os.path.exists(newpath_main):
    os.makedirs(newpath_main)   
os.chdir(newpath_main)
os.getcwd()

'C:\\MDE_Data'

In [13]:
EthnicityGender = r'C:\MDE_Data\Enrollment_EthnicityGender'
if not os.path.exists(EthnicityGender):
    os.makedirs(EthnicityGender)
    
os.chdir(EthnicityGender)
os.getcwd()

'C:\\MDE_Data\\Enrollment_EthnicityGender'

In [14]:
# Find all Enrollment by Ethnicity/Gender datasets
TReport_href_Enrollment_by_EthnicityGender = driver.find_elements_by_xpath('//a[@href and contains(@title,"Enrollment by Ethnicity/Gender")]')

In [15]:
# Create a rolling window object to select a subset of files to download.
CurrentTime = datetime.datetime.now()
ThisYearFile = str(CurrentTime.year - 1) + '-'
LastYearFile = str(CurrentTime.year - 2) + '-'

In [16]:
TReport_href_Enrollment_by_EthnicityGender[0].get_attribute('title')

'Data file: 2017-18 Enrollment by Ethnicity/Gender'

In [17]:
 # All Enrollment by Ethnicity/Gender datasets
downloadsTitle = []
downloadURL = []
for ii in TReport_href_Enrollment_by_EthnicityGender:
    if ThisYearFile in ii.get_attribute('title'):
        ThisYearFileP = wget.download(ii.get_attribute('href'))
    if LastYearFile in ii.get_attribute('title'):   
        LastYearFileP = wget.download(ii.get_attribute('href')) 

100% [..........................................................................] 3344268 / 3344268

In [18]:
os.chdir(EthnicityGender)
ThisYearPublic = pd.read_excel(ThisYearFileP, sheetname = 'School')
LastYearPublic = pd.read_excel(LastYearFileP, sheetname = 'School')

In [19]:
# function for subsetting the public and non-public hs enrollment data
def SchoolConditionalJuniors(DataSet):
    columnsOfInterest = ['districtNumber', 'districtType', 'schoolNumber', 
                         'EconomicDevelopmentRegion','SchoolName', 
                         'TotalMinority', 'TotalStudents', 'Grade']
    return DataSet[columnsOfInterest][(DataSet.Grade == '11') &
                                      (DataSet.TotalStudents > 0)]

In [20]:
# function for subsetting the public and non-public hs enrollment data
def SchoolConditionalSeniors(DataSet):
    columnsOfInterest = ['districtNumber', 'districtType', 'schoolNumber', 
                         'EconomicDevelopmentRegion',
                         'SchoolName', 'TotalMinority', 
                         'TotalStudents', 'Grade']
    return DataSet[columnsOfInterest][(DataSet.Grade == '12') &
                                      (DataSet.TotalStudents > 0)]

In [21]:
JuniorsThsYear = SchoolConditionalJuniors(ThisYearPublic)
JuniorsThsYear = JuniorsThsYear.rename(columns = {'TotalStudents': 'JuniorsThisYear',
                                                 'TotalMinority': 'JrMinorityThisYear'})
JuniorsLstYear = SchoolConditionalJuniors(LastYearPublic)
JuniorsLstYear = JuniorsLstYear.rename(columns = {'TotalStudents': 'JuniorsLastYear',
                                                 'TotalMinority': 'JrMinorityLastYear'})
JuniorsPublic = pd.merge(JuniorsThsYear, JuniorsLstYear, how = 'inner',
                         on = ['districtNumber', 'districtType', 
                               'schoolNumber', 'SchoolName',
                              'EconomicDevelopmentRegion'])

In [22]:
JuniorsPublic = JuniorsPublic[['districtNumber', 'districtType', 'schoolNumber', 
                               'SchoolName', 'EconomicDevelopmentRegion',
                               'JuniorsThisYear', 'JrMinorityThisYear', 
                               'JuniorsLastYear', 'JrMinorityLastYear']]

In [23]:
SeniorsThsYear = SchoolConditionalSeniors(ThisYearPublic)
SeniorsThsYear = SeniorsThsYear.rename(columns = {'TotalStudents': 'SeniorsThisYear',
                                                 'TotalMinority': 'SrMinorityThisYear'})
SeniorsLstYear = SchoolConditionalSeniors(LastYearPublic)
SeniorsLstYear = SeniorsLstYear.rename(columns = {'TotalStudents': 'SeniorsLastYear',
                                                 'TotalMinority': 'SrMinorityLastYear'})
SeniorsPublic = pd.merge(SeniorsThsYear, SeniorsLstYear, how = 'inner',
                        on = ['districtNumber', 'districtType', 
                              'schoolNumber', 'SchoolName',
                              'EconomicDevelopmentRegion'])

In [24]:
SeniorsPublic = SeniorsPublic[['districtNumber', 'districtType', 'schoolNumber', 
                               'SchoolName', 'EconomicDevelopmentRegion',
                               'SeniorsThisYear', 'SrMinorityThisYear', 
                               'SeniorsLastYear', 'SrMinorityLastYear']]

In [25]:
TargetDF = pd.merge(JuniorsPublic, SeniorsPublic, 
                    how = 'inner', on = ['districtNumber', 
                                         'districtType',
                                         'schoolNumber', 
                                         'EconomicDevelopmentRegion',
                                         'SchoolName'])

In [26]:
TargetDF.head()

,districtNumber,districtType,schoolNumber,SchoolName,EconomicDevelopmentRegion,JuniorsThisYear,JrMinorityThisYear,JuniorsLastYear,JrMinorityLastYear,SeniorsThisYear,SrMinorityThisYear,SeniorsLastYear,SrMinorityLastYear
0,1.0,1.0,1.0,AITKIN SECONDARY SCHOOL,03,86.0,8.0,81.0,7.0,82.0,6.0,94.0,6.0
1,1.0,1.0,6.0,AITKIN ALTERNATIVE LEARNING PROGRAM,03,1.0,0.0,1.0,0.0,5.0,1.0,2.0,0.0
2,1.0,3.0,347.0,FAIR Senior High,11,52.0,38.0,71.0,39.0,65.0,37.0,63.0,39.0
3,1.0,3.0,348.0,MERC,11,5.0,3.0,9.0,9.0,46.0,41.0,49.0,43.0
4,1.0,3.0,349.0,LORING-NICOLLET HIGH,11,15.0,11.0,11.0,9.0,37.0,24.0,33.0,24.0


In [31]:
# GET MDE Lookup Table

In [30]:
wget.download('http://w20.education.state.mn.us/MdeOrgView/tag/extractContacts/MDEORG_DISTRICT_SCHOOL?description=')
filename =  'School_' + datetime.datetime.today().strftime('%m-%d-%Y') + '.csv'
LookUp = pd.read_csv(filename)
LookUpColumns = ['District Number', 'District Type', 'School Number','Physical Line 1',  'Physical Line 2',
       'Physical City', 'Physical State', 'Physical Zip','County','ACT ID']

-1 / unknown

In [31]:
MNp = pd.merge(LookUp[LookUpColumns],TargetDF,  how = "inner"
                  , right_on = ['districtNumber', 'districtType', 'schoolNumber']
                  , left_on = ['District Number', 'District Type', 'School Number']
                   ).drop(['districtNumber', 
                           'districtType', 
                           'schoolNumber',
                           'District Number', 
                           'District Type', 
                           'School Number'],
                          axis = 1).drop_duplicates()

In [33]:
MNp = MNp[['Physical Line 1',  'Physical Line 2',
       'Physical City', 'Physical State', 
    'Physical Zip','County','ACT ID','EconomicDevelopmentRegion',
'SeniorsThisYear', 'SeniorsLastYear', 'SrMinorityThisYear', 'SrMinorityLastYear',
'JuniorsThisYear', 'JuniorsLastYear', 'JrMinorityThisYear', 'JrMinorityLastYear']]

In [34]:
MNp['PhysicalZipSuff'] = MNp['Physical Zip'].str.split('-').str[1]

In [35]:
MNp['Physical Zip'] = MNp['Physical Zip'].str.split('-').str[0]

In [36]:
MNp = MNp.rename(columns = {'District Number': 'DistrictNumber', 
       'District Type': 'DistrictType', 
       'School Number': 'SchoolNumber',
       'Physical Line 1': 'PhysicalLine1',  
       'Physical Line 2': 'PhysicalLine2',
       'Physical City': 'PhysicalCity', 
       'Physical State': 'PhysicalState',
       'Physical Zip': 'PhysicalZip',
       'ACT ID': 'ACTID', 
       'Data Extracted': 'DataExtracted'})

In [38]:
MNp.to_csv("MnPublic.csv", index = False)

In [40]:
len(MNp[MNp['ACTID'].isnull() == True])

167

In [41]:
len(MNp)

690

In [43]:
len(MNp[MNp['ACTID'].isnull() == False])

523

In [34]:
# Pre-Lookup missing ACT

In [35]:
ActLookupRequest = missingACT[['SchoolName', 'Physical City']]

In [36]:
State = 'Minnesota'

In [37]:
ActLookupRequest['State'] = State

C:\Users\karlk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
actURL = 'https://www.act.org/content/act/en/products-and-services/the-act/registration/high-school-codes-lookup.html'

In [45]:
ActLookupRequest = ActLookupRequest.reset_index(drop = True)

In [46]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
import pyautogui
from random import randint
import re
#TODO navigate browser to actURL

In [58]:
for i in range(len(ActLookupPublicRequest)):
    pyautogui.click(button='left', x=500, y=500)
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    pyautogui.typewrite('Minnesota')
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    pyautogui.press('tab')
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    pyautogui.typewrite(ActLookupPublicRequest['Physical City'].iloc[i]) 
    pyautogui.press('tab')  
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    #original
    #pyautogui.typewrite(ActLookupRequest.SchoolName.[i])
    pyautogui.typewrite(ActLookupPublicRequest.SchoolName.iloc[i])
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    pyautogui.press('tab')    
    pyautogui.press('enter')
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    time.sleep(1)
    try:
        time.sleep(1)
        time.sleep(1)
        time.sleep(1)
        xpath = driver.find_element_by_xpath(("//ul[@class='center-code']"))
        actcode = xpath.text
        import re
        actcode = re.sub("[^0-9]", "",actcode)
        act.append(actcode)
        sn.append(ActLookupPublicRequest.SchoolName.iloc[i])
        city.append(ActLookupPublicRequest['Physical City'].iloc[i])
        time.sleep(1)
        driver.get(actURL)
    except:
        
        time.sleep(1)
        driver.get(actURL)
        continue 
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
ACTPublicFound = pd.DataFrame({'ACT ID': act,
        'SchoolName': sn,
        'Physical City': city})
#ACTPublicFound = FinalMDEPublicACT 
PublicLookupUpdate = pd.merge(ActLookupPublicRequest, ACTPublicFound
                              , on = ['SchoolName', 'Physical City']
                              , how= 'inner')

In [ ]:
PublicLookupUpdate.to_csv("HS_ACT_FOUND.csv", index = False)